# Butina Clustering
The Butina algorithm as first described in 1999 ([*J. Chem. Inf. Comput. Sci.* **1999**, *39*, 747-750](https://pubs.acs.org/doi/10.1021/ci9803381)) specifically for clustering molecules. 

## Imports & Settings

In [1]:
### Imports
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
### Add the utils directory to the path
sys.path.append(os.path.abspath("../utils"))


# Data
The chemical structures of BRD4 inhibitors are encoded in bit vectors. For this data set **Morgan fingerprints** `rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect` was used. 






In [3]:
### Load the data
df = pd.read_pickle("data/morgan_2048_df.pkl")

df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
ID,,,,,,,,,,,,,,,,,,,,,
CHEMBL1232461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1233528,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1313432,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1344420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1361699,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMBL5440963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tanimoto Similarity

The Tanimoto similarity will be used as distance metric. Between two binary fingerprints **A** and **B** it is defined as:

$$
\text{Tanimoto}(A, B) = \frac{|A \cap B|}{|A| + |B| - |A \cap B|}
$$

Where:
- $ |A \cap B| $ is the number of bits that are 1 in both A and B (i.e., the intersection)
- $ |A| $ is the number of bits that are 1 in A
- $ |B| $ is the number of bits that are 1 in B

![Tanimoto Similarity Venn Diagram](images/tanimoto_venn.svg)

Identical molecules have $\text{Tanimoto}(A, A) = 1$

Very dissimilar molecules have $\text{Tanimoto}(A, B) = 0$

# Butina Algorithm
1) The only hyperparameter is the **Tanimoto threshold**.
2) All compounds within this threshold are considered **neighbors**. 
3) The compound with the most neighbors is defined as the **centroid** of the first cluster. All its neighbors are cluster members.
4) Compounds assigned to a cluster (both as centroid or cluster member) are ignored for defining the next cluster. Step 3 is repeated until all compounds with neighbors have been assigned to a cluster.
5) Compounds not assigned to a cluster are defined as **singletons**. 


Demonstrative example for Butina clustering

![Butina](images/butina.svg)

- The Butina algorithm is deterministic.
- Singletons can be within the defined Tanimoto threshold of compounds assigned to a centroid with more neighbors.
- The assumption of this method is that the compound with the most neighbors within a cluster - the centroid - best represents all cluster features. 
- Because the number of clusters is not pre-defined and all cluster members are within the Tanimoto threshold of its centroid clusters tend to be more homogenous compared to other methods. This is because chemical data sets are often distributed very unevenly.  



# Math
To calculate the intersection between two fingerprints boolean and integers were considered. 

In [4]:
demo_df = pd.DataFrame({
    "ID": ["F1", "F2", "F3", "F4", "F5"],
    "bit_0": [1, 0, 1, 0, 1],
    "bit_1": [1, 0, 1, 1, 0],
    "bit_2": [0, 1, 0, 1, 0],
    "bit_3": [1, 0, 1, 0, 1],
}).set_index("ID")
demo_df

,bit_0,bit_1,bit_2,bit_3
ID,,,,
F1,1,1,0,1
F2,0,0,1,0
F3,1,1,0,1
F4,0,1,1,0
F5,1,0,0,1


In [5]:
X_int = demo_df.to_numpy().astype(int)
intersection_matrix = X_int @ X_int.T
pd.DataFrame(intersection_matrix, index=demo_df.index, columns=demo_df.index)

ID,F1,F2,F3,F4,F5
ID,,,,,
F1,3,0,3,1,2
F2,0,1,0,1,0
F3,3,0,3,1,2
F4,1,1,1,2,0
F5,2,0,2,0,2


In [6]:
X_bool = demo_df.to_numpy().astype(bool)
intersection_matrix = np.bitwise_and(X_bool[:, None, :], X_bool[None, :, :]).sum(axis=2)
pd.DataFrame(intersection_matrix, index=demo_df.index, columns=demo_df.index)

ID,F1,F2,F3,F4,F5
ID,,,,,
F1,3,0,3,1,2
F2,0,1,0,1,0
F3,3,0,3,1,2
F4,1,1,1,2,0
F5,2,0,2,0,2


In [9]:
class Butina:
    """
    Butina clustering algorithm for fingerprints.

    Attributes
    ----------
    threshold : float
        Similarity threshold for clustering.
    df : pd.DataFrame
        Input DataFrame with the fingerprints.
    similarity_df : pd.DataFrame
        DataFrame with the similarity matrix.
    cluster_df : pd.DataFrame
        DataFrame with the cluster assignments. Same length and index as df.
    """

    def __init__(self, threshold=0.75):
        self.threshold = threshold
        self.df = None
        self.similarity_df = None
        self.cluster_df = None


    def fit(self, df):
        """
        Compute the similarity matrix and assign all compounds to clusters.
        
        Parameters
        ----------
        df : pd.DataFrame
            DataFrame with the fingerprints. Requires the index to be unique.
        
        Returns
        -------
        cluster_df : pd.DataFrame
            DataFrame with the cluster assignments. Same length and index as df.
        
        Notes
        -----        
        - Calls _compute_similarity_matrix() to compute the similarity matrix.
        - Calls _assign_next_cluster() to assign clusters in a loop.
        
        """

        ### Check if the input DataFrame has unique indexes
        if not df.index.is_unique:
            raise ValueError("Input DataFrame indexes must be unique.")
        
        ### Assigne the input DataFrame to the instance
        self.df = df.copy()

        ### Instantiate the DataFrame to store the cluster assignments with NaN values
        self.cluster_df = pd.DataFrame(index=df.index, columns=["Cluster"])
        self.cluster_df['Centroid'] = False
        self.cluster_df['Singleton'] = False

        ### Verbose output
        print("Similarity matrix computation")

        ### Compute the similarity matrix
        self._compute_similarity_matrix()

        ### Verbose output
        print("Clustering")

        ### Progress bar
        with tqdm(total=100) as pbar:

            ### Loop until all compounds are assigned to a cluster
            while self.cluster_df["Cluster"].isna().any():
                
                ### Get the next cluster members    
                cluster_members = self._assign_next_cluster()
                
                ### Update the progress bar
                pbar.update(cluster_members / len(self.df) * 100)

        return self.cluster_df



    def _assign_next_cluster(self):
        """
        Define the next cluster by identifying the centroid (compound with most neighbors) and its neighbors.

        Returns
        -------
        int
            The number of assigned cluster members.
        
        Side Effects
        ------------
        - Updates the cluster_df DataFrame with the cluster assignments.         
        """
        
        ### Define the ID of the next cluster (starting from 0)
        last_cluster_id = self.cluster_df['Cluster'].max()
        next_cluster_id = 0 if pd.isna(last_cluster_id) else int(last_cluster_id + 1)

        ### Subset the similarity matrix for all compounds without cluster assignment
        unassigned_mask = self.cluster_df["Cluster"].isna()
        free_sim_df = self.similarity_df.loc[unassigned_mask, unassigned_mask]           

        ### Identify the centroid of the next cluster (the compound with most neighbors)
        centroid_id = (free_sim_df > self.threshold).sum(axis=1).idxmax()

        ### Identify the cluster members (neighbors of the centroid)
        neigb_id = free_sim_df.loc[free_sim_df[centroid_id] > self.threshold].index

        ### Assign the cluster ID to the centroid and its neighbors
        self.cluster_df.loc[centroid_id, "Cluster"] = next_cluster_id
        self.cluster_df.loc[neigb_id, "Cluster"] = next_cluster_id

        ### Assign the Centroid flag 
        self.cluster_df.loc[centroid_id, "Centroid"] = True

        ### Assign the Singleton flag to the centroid if it has no neighbors
        if len(neigb_id) == 0:
            self.cluster_df.loc[centroid_id, "Singleton"] = True
        
        ### Return the number of assigned cluster members
        return len(neigb_id) + 1


    def _compute_similarity_matrix(self):
        """
        Compute the similarity matrix for the fingerprints in the DataFrame.
        
        Notes
        -----
        The similarity is computed as the Tanimoto index, which is defined as the size of the intersection divided by the size of the union of two sets.
        """
        

        ### Convert the DataFrame with fingerprints to a NumPy array
        X = self.df.to_numpy().astype(int)

        ### Compute the intersection of the fingerprints
        intersect = X @ X.T

        ### Compute the number of on bits in each fingerprint
        on_bits = X.sum(axis=1)

        ### Compute the denominator
        denom = on_bits[:, None] + on_bits[None, :] - intersect
        
        ### Compute the distance matrix
        sim = np.divide(intersect, denom, out=np.zeros_like(intersect, dtype=float), where=denom != 0)

        ### Set the diagonal to 0
        np.fill_diagonal(sim, 0)

        ### Set the similarity matrix
        self.similarity_df = pd.DataFrame(sim, index=self.df.index, columns=self.df.index)




In [10]:
butina = Butina(threshold=0.75)
butina.fit(df.sample(1000))

Similarity matrix computation
Clustering


100%|█████████▉| 99.99999999999844/100 [00:05<00:00, 19.71it/s] 


,Cluster,Centroid,Singleton
ID,,,
CHEMBL3892997,105,True,True
CHEMBL4061600,5,True,False
CHEMBL4790841,106,True,True
CHEMBL4163715,107,True,True
CHEMBL4748580,13,True,False
...,...,...,...
CHEMBL4080510,44,False,False
CHEMBL5191078,830,True,True
CHEMBL5174374,22,False,False


In [126]:
empty_index = butina.cluster_df[butina.cluster_df['Singleton'] > 10].index
len(empty_index)

0

In [119]:
butina.cluster_df.sort_values("Cluster",ascending=False).head(20)

,Cluster,Centroid,Singleton
ID,,,
CHEMBL4204065,839,True,False
CHEMBL3984289,838,True,False
CHEMBL4159382,837,True,False
CHEMBL3814896,836,True,False
CHEMBL5218934,835,True,False
CHEMBL3985294,834,True,False
CHEMBL4465676,833,True,False
CHEMBL2346691,832,True,False
CHEMBL3977798,831,True,False


# Sandbox

In [100]:
butina.cluster_df.dropna()

,Cluster,Centroid,Singleton
ID,,,
CHEMBL3753333,0,False,False
CHEMBL4078384,0,False,False
CHEMBL4085329,0,False,False
CHEMBL4077279,0,False,False
CHEMBL4086937,0,False,False
CHEMBL4075476,0,False,False
CHEMBL4075074,0,False,False
CHEMBL4066476,0,False,False
CHEMBL4063726,0,False,False


In [101]:
(butina.similarity_df > 0.75).sum(axis=1).sort_values(ascending=False)

ID
CHEMBL4084580    14
CHEMBL4066476    12
CHEMBL5403358    12
CHEMBL3753333    11
CHEMBL5419367    11
                 ..
CHEMBL3220925     0
CHEMBL4755816     0
CHEMBL5094520     0
CHEMBL5414895     0
CHEMBL3963861     0
Length: 2000, dtype: int64

In [52]:
(butina.similarity_df > 0.75).sum(axis=1).sort_values(ascending=False).iloc[2:4].idxmax()

'CHEMBL4745597'

In [95]:
mask = butina.similarity_df.loc[butina.similarity_df['CHEMBL3356565'] > 0.5].index

type(butina.cluster_df.loc[mask].iloc[0,0])

float

In [86]:
if butina.cluster_df['Cluster'].max():
    print('Oskar')

Oskar


In [91]:
butina.cluster_df

,Cluster,Centroid
ID,,
CHEMBL3356565,1,1
CHEMBL4442201,1,1
CHEMBL3398330,1,1
CHEMBL4080905,1,1
CHEMBL4073350,1,1
...,...,...
CHEMBL5402704,NaN,NaN
CHEMBL4206831,NaN,NaN
CHEMBL3650881,NaN,NaN


In [93]:
butina.similarity_df.loc[butina.cluster_df["Cluster"].isna(), butina.cluster_df["Cluster"].isna()]

ID,CHEMBL4225700,CHEMBL5397122,CHEMBL4211379,CHEMBL4790613,CHEMBL3974456,CHEMBL3780115,CHEMBL3398332,CHEMBL4206026,CHEMBL4213395,CHEMBL4752747,...,CHEMBL3898472,CHEMBL5290468,CHEMBL4798570,CHEMBL4062259,CHEMBL4169300,CHEMBL5402704,CHEMBL4206831,CHEMBL3650881,CHEMBL3654296,CHEMBL4204922
ID,,,,,,,,,,,,,,,,,,,,,
CHEMBL4225700,0.000000,0.088710,0.110000,0.071429,0.050505,0.134146,0.130952,0.153846,0.110000,0.146789,...,0.075472,0.100000,0.137097,0.089431,0.130952,0.080292,0.103448,0.141304,0.112903,0.094737
CHEMBL5397122,0.088710,0.000000,0.123288,0.083333,0.090278,0.140625,0.080292,0.108974,0.115646,0.112500,...,0.091503,0.075581,0.102273,0.126506,0.129771,0.092391,0.117284,0.074830,0.091429,0.097902
CHEMBL4211379,0.110000,0.123288,0.000000,0.043860,0.055556,0.099099,0.087719,0.120301,0.443299,0.140741,...,0.091603,0.087838,0.089744,0.124138,0.087719,0.180000,0.121429,0.107438,0.098684,0.445652
CHEMBL4790613,0.071429,0.083333,0.043860,0.000000,0.108911,0.160920,0.061856,0.075630,0.072072,0.108333,...,0.129630,0.076923,0.071942,0.067669,0.107527,0.075862,0.096774,0.076190,0.065693,0.046729
CHEMBL3974456,0.050505,0.090278,0.055556,0.108911,0.000000,0.055046,0.063636,0.083969,0.064000,0.072993,...,0.214286,0.115942,0.051613,0.090909,0.054054,0.069182,0.071429,0.067227,0.052632,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMBL5402704,0.080292,0.092391,0.180000,0.075862,0.069182,0.081633,0.087838,0.100592,0.187919,0.130178,...,0.090909,0.093923,0.106952,0.129944,0.066225,0.000000,0.204969,0.068750,0.108696,0.164384
CHEMBL4206831,0.103448,0.117284,0.121429,0.096774,0.071429,0.120968,0.155738,0.077922,0.145985,0.163265,...,0.088435,0.119497,0.126506,0.104294,0.101562,0.204969,0.000000,0.102190,0.082353,0.119403
CHEMBL3650881,0.141304,0.074830,0.107438,0.076190,0.067227,0.074074,0.063063,0.075188,0.089431,0.080292,...,0.104839,0.076389,0.115646,0.090278,0.072727,0.068750,0.102190,0.000000,0.175182,0.094828


In [84]:
butina.similarity_df[mask]

ID,CHEMBL3356564,CHEMBL3356554,CHEMBL3356552
ID,,,
CHEMBL3356565,0.670886,0.682353,0.588889
CHEMBL4442201,0.198276,0.190476,0.181102
CHEMBL3398330,0.060345,0.098361,0.063492
CHEMBL4080905,0.066667,0.068750,0.055556
CHEMBL4073350,0.071429,0.091667,0.065041
...,...,...,...
CHEMBL5402704,0.069182,0.103659,0.064706
CHEMBL4206831,0.111111,0.141844,0.102740
CHEMBL3650881,0.067227,0.069767,0.061538


In [70]:
butina.cluster_df

,Cluster,Centroid
ID,,
CHEMBL3356565,NaN,NaN
CHEMBL4442201,NaN,NaN
CHEMBL3398330,NaN,NaN
CHEMBL4080905,NaN,NaN
CHEMBL4073350,NaN,NaN
...,...,...
CHEMBL5402704,NaN,NaN
CHEMBL4206831,NaN,NaN
CHEMBL3650881,NaN,NaN
